### **Предобработка данных**

На данном этапе происходит **структуризация данных** из документов. Этот процесс можно назвать "атомизацией" или "разбиением на элементы", но чаще всего его относят к **предобработке данных**, поскольку полученные данные будут использоваться в дальнейшем в **векторизации** для работы с моделями LLM (Large Language Models).

#### **Цель этапа:**
Извлечь содержимое документов, сохранив их **иерархическую структуру**, что позволяет представить данные в удобном формате для дальнейшего анализа, поиска или обработки.


### **Пример результата обработки**

Каждый документ разбивается на **атомарные единицы содержания**, которые затем сохраняются с учетом их структуры. Например, исходный документ `Книга_1 по Дону` (для примера взяла первый раздел):

**`1_Физико-географическое_описание_речного_бассейна.pdf`**

будет разделен на следующие части:
```plaintext
1_Физико-географическое_описание_речного_бассейна
├── 1.1_Территория
│   └── 1.1_Территория.pdf
├── 1.2_Рельеф._Гидрография
│   └── 1.2_Рельеф._Гидрография.pdf
├── 1.3_Геоморфология
│   └── 1.3_Геоморфология.pdf
├── 1.4_Почвенный_покров
│   └── 1.4_Почвенный_покров.pdf
├── 1.5_Климат
│   └── 1.5_Климат.pdf
└── 1_Физико-географическое_описание_речного_бассейна.pdf
```

#### **Пример выделенной главы:**
- **1.1 Территория**: Содержит описание географических особенностей территории.
  - Файл: `1.1_Территория.pdf`

- **1.2 Рельеф и гидрография**: Включает данные о рельефе местности и характеристиках гидрографии.
  - Файл: `1.2_Рельеф._Гидрография.pdf`

- **1.3 Геоморфология**: Подробный разбор геоморфологических процессов.
  - Файл: `1.3_Геоморфология.pdf`

### **Результат:**
1. **Сохранена структура документа** — каждая глава и подглава (в приницпе уровень не меет значения, хоть до 5 уровня подглавы) имеют собственный файл (PDF).
3. **Удобство навигации по главам в формате PDF** — иерархическая структура позволяет легко находить нужные разделы.
4. **Единая структура в JSON** — для дальнейшей работы с текстом все данные хранятся в удобной иерархической структуре в формате JSON. Для каждого отчета свой JSON.

In [22]:
# библиотеки
import fitz
import json
import os

# самописные функции
import libs as lb

# Сохранение иерархии в JSON-файл
def save_hierarchy_to_json(hierarchy, output_path):
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(hierarchy, json_file, ensure_ascii=False, indent=4)

### Пример на одном отчете по Дону
(результаты сохранены в `notebooks/data_preprocessing/pdf_json_decomposer/outputs`)

In [23]:
def process_pdf(pdf_path, base_path, output_json_name):
    """
    Основной процесс обработки PDF файла:
    1. Извлечение оглавления.
    2. Построение иерархии разделов.
    3. Обновление конечных страниц разделов.
    4. Извлечение текста из разделов.
    5. Сохранение структуры в JSON.
    6. Создание директории и сохранение PDF разделов.

    :param pdf_path: Путь к PDF файлу.
    :param base_path: Базовый путь для сохранения выходных данных.
    :param output_json_name: Имя JSON файла для сохранения иерархии.
    """
    try:
        # Открываем PDF
        print(f"Открытие PDF файла: {pdf_path}")
        doc = fitz.open(pdf_path)

        # Разделение на секции и построение иерархии
        print("Извлечение оглавления и построение иерархии...")
        sections = lb.split_pdf_by_toc(pdf_path)
        hierarchy = lb.build_hierarchy(sections)

        # Обновление конечных страниц для всех секций
        print("Обновление конечных страниц секций...")
        lb.update_all_end_pages(hierarchy)

        # Извлечение текста из всех секций
        print("Извлечение текста из разделов...")
        for section in hierarchy:
            lb.attach_text_to_deepest_sections(section, doc)

        # Создание директории и структуры файлов
        print("Создание структуры директорий и сохранение PDF...")
        lb.create_directory_structure(hierarchy, base_path, pdf_path)

        # Сохранение иерархии в JSON
        json_output_path = os.path.join(base_path, output_json_name)
        print(f"Сохранение иерархии в JSON файл: {json_output_path}")
        save_hierarchy_to_json(hierarchy, json_output_path)

        print(f"Обработка завершена. Данные сохранены в: {base_path}")

    except Exception as e:
        print(f"Произошла ошибка при обработке PDF: {e}")
    finally:
        # Закрываем PDF документ
        doc.close()

In [24]:
# Основной вызов функции
if __name__ == "__main__":
    pdf_path = "../../../data/Base_Books/Дон/Книга 1/Книга_1.pdf"
    base_path = "outputs"
    output_json_name = "Книга_1.json"

    process_pdf(pdf_path, base_path, output_json_name)

Открытие PDF файла: ../../../data/Base_Books/Дон/Книга 1/Книга_1.pdf
Извлечение оглавления и построение иерархии...
Обновление конечных страниц секций...
Проверка раздела: ВВЕДЕНИЕ, start_page: 9, end_page: 9
Проверка раздела: 1 Физико-географическое описание речного бассейна, start_page: 10, end_page: 44
Проверка раздела: 1.1 Территория, start_page: 10, end_page: 11
Проверка раздела: 1.2 Рельеф. Гидрография, start_page: 12, end_page: 14
Проверка раздела: 1.3 Геоморфология, start_page: 15, end_page: 20
Проверка раздела: 1.4 Почвенный покров, start_page: 21, end_page: 35
Проверка раздела: 1.5 Климат, start_page: 36, end_page: 44
Проверка раздела: 2 СОЦИАЛЬНО-ЭКОНОМИЧЕСКАЯ ХАРАКТЕРИСТИКА ТЕРРИТОРИИ РЕЧНОГО БАССЕЙНА, start_page: 45, end_page: 63
Проверка раздела: 2.1 Прогноз основных показателей долгосрочного социально-экономического развития территории бассейна р.Дон до 2020 г., start_page: 53, end_page: 63
Проверка раздела: 3 ПОВЕРХНОСТНЫЕ ВОДНЫЕ РЕСУРСЫ, start_page: 64, end_page: 93
Пр

### Автоматический запуск для всех бассейнов из директории `data/BaseBooks`
(результаты сохранены на сервере, тут не приведены так  как вес ~ 60 ГБ)

In [25]:
def process_all_pdfs(base_dir, output_base_dir):
    """
    Проходит по всей директории `base_dir`, обрабатывает все PDF файлы и сохраняет результаты в `output_base_dir`.
    """
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".pdf"):
                pdf_path = os.path.join(root, file)

                # Создание соответствующей структуры директорий для выхода
                relative_path = os.path.relpath(root, base_dir)
                output_dir = os.path.join(output_base_dir, relative_path)
                os.makedirs(output_dir, exist_ok=True)

                # Генерация имени JSON файла
                json_name = os.path.splitext(file)[0] + ".json"

                # Обработка PDF
                process_pdf(pdf_path, output_dir, json_name)

In [26]:
if __name__ == "__main__":
    base_dir = "../../../data/Base_Books"
    output_base_dir = "outputs_all"

    print(f"Начало обработки всех PDF файлов в директории: {base_dir}")
    process_all_pdfs(base_dir, output_base_dir)
    print("Обработка завершена.")

Начало обработки всех PDF файлов в директории: ../../../data/Base_Books
Открытие PDF файла: ../../../data/Base_Books/Печора/Книга 3/Книга_3.pdf
Извлечение оглавления и построение иерархии...
Обновление конечных страниц секций...
Извлечение текста из разделов...
Создание структуры директорий и сохранение PDF...
Сохранение иерархии в JSON файл: outputs_all/Печора/Книга 3/Книга_3.json
Обработка завершена. Данные сохранены в: outputs_all/Печора/Книга 3
Открытие PDF файла: ../../../data/Base_Books/Печора/Книга 4/Книга_4.pdf
Извлечение оглавления и построение иерархии...
Обновление конечных страниц секций...
Извлечение текста из разделов...
Создание структуры директорий и сохранение PDF...
Сохранение иерархии в JSON файл: outputs_all/Печора/Книга 4/Книга_4.json
Обработка завершена. Данные сохранены в: outputs_all/Печора/Книга 4
Открытие PDF файла: ../../../data/Base_Books/Печора/Книга 5/Книга_5.pdf
Извлечение оглавления и построение иерархии...
Обновление конечных страниц секций...
Извлечени